# Tracking Resource Usage with PennyLane Device Tracker

In this notebook we will explore how to use the PennyLane device tracker feature with Amazon Braket.
As is demonstrated in the [optimization of quantum circuits notebook](../1_Parallelized_optimization_of_quantum_circuits/1_Parallelized_optimization_of_quantum_circuits.ipynb), computing gradients of quantum circuits involves multiple devices executions.
This can lead to a large number of executions when optimizing quantum circuits.
So to help users keep track of their usage, Amazon Braket works with PennyLane to record and make available useful information during the computation.
The PennyLane device resource tracker keeps a record of the usage of a device, such as numbers of circuit evaluations and shots.
Amazon Braket extends this information with task IDs and simulator duration to allow further tracking.
The device tracker can be combined with additional logic to monitor and limit resource usage on devices.

## Setup

In [1]:
import pennylane as qml
from pennylane import numpy as np

import time

Before we show the device resource tracker, let's first set up a circuit for demonstration and experimenting.
We will use a simple two qubit circuit with two parameters throughout this notebook.
We will start by evaluating it using the local Braket simulator.

In [2]:
wires = 2  # Number of qubits

dev = qml.device("braket.local.qubit", wires=wires)

def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

qnode_local = qml.QNode(circuit, dev)
params = np.array([0.1, 0.2], requires_grad=True)

## The PennyLane Device Resource Tracker

The PennyLane device resource tracker is a python context manager created by `qml.Tracker`.
To use the device tracker to track a single evaluation of our simple circuit, we put the evaluation inside a `with` statement block.
Only evaluations inside of the `with` block will have their usage recorded and accumulated.

In [3]:
with qml.Tracker(dev) as tracker:
    print("Expectation value of circuit:", qnode_local(params))

Expectation value of circuit: 0.9751703272018161


With this execution complete, all of the recorded information is available in `tracker`.
There are three interfaces to access the data inside of the resource tracker.
The full history of each update is available through `tracker.history`.
Numerical values are accumulated, and each of their totals are in `tracker.totals`.
And lastly, the most recent update to the tracker is kept in `tracker.latest`.

Let's first look at the history of what was recorded by evaluating our circuit.

In [4]:
print(tracker.history)

{'executions': [1], 'shots': [None], 'braket_task_id': ['3e22ac39-a7e7-425d-af6a-1d4f3cc7974f'], 'batches': [1], 'batch_len': [1]}


We can see that this evaluation lead to a single circuit execution.
This single execution had no shots, and was evaluated with a single batch of length 1.
When using the Braket local simulator, the device tracker records the unique id that the simulator assigns to new tasks.

Next, let's evaluate the gradient of this circuit and track the device usage.

In [6]:
with qml.Tracker(dev) as tracker:
    print("Gradient of circuit:", qml.grad(qnode_local)(params))

print(tracker.history)

Gradient of circuit: [-0.0978434  -0.19767681]
{'executions': [1, 1, 1, 1, 1], 'shots': [None, None, None, None, None], 'braket_task_id': ['7602e62c-dd42-40bf-a214-147fdeec9c93', '9e924369-432e-4e86-bd28-c18a3bd7ea2d', 'fe68b242-166c-496a-b1e2-df72bc7eedc6', '8e1f1427-6b2a-4261-a229-dec502fc008f', 'c6426b8f-6b25-4c9f-abac-ad60ae96c4c8'], 'batches': [1, 1], 'batch_len': [1, 4]}


Recall that evaluating the gradient of a quantum circuit will result in multiple circuit evaluations.
Here we see that to calculate this gradient, 5 new tasks were created and recorded in the device tracker.
For values which are numeric, such as the `executions`, the device tracker will accumulate all of the recorded data into `tracker.totals`

In [7]:
print(tracker.totals)

{'executions': 5, 'batches': 2, 'batch_len': 5}


Note that `None` is not numeric, which is why the shots are not accumulated in `tracker.totals` in the previous example.
If we instead compute the gradient with a finite number of shots, we will see the total shots recorded.
Let's try the gradient with `shots=100`.
Since there are 5 circuit evaluations, we should expect the total shots to be `500`. 

In [8]:
with qml.Tracker(dev) as tracker:
    qml.grad(qnode_local)(params, shots=100)
print(tracker.totals)

{'executions': 5, 'shots': 500, 'batches': 2, 'batch_len': 5}


We can also inspect the most recently recorded data with `tracker.latest`.
Note that not every field will be present in every update.
The latest update in this gradient calculation only included `'batches'` and `'batch_len'`.

In [9]:
print(tracker.latest)

{'batches': 1, 'batch_len': 4}


## Using Amazon Braket Simulators

The Amazon Braket on-demand simulators report additional information to the device tracker.
Let's set up a remote device using Amazon Braket SV1, and create a new `QNode` to run our circuit on the on-demand simulator.

In [11]:
device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

dev_remote = qml.device(
    "braket.aws.qubit",
    device_arn=device_arn,
    wires=wires
)

qnode_remote = qml.QNode(circuit, dev_remote)

Now we can send the same simple circuit from above to the AWS on-demand simulator and track the resource usage.

In [12]:
with qml.Tracker(dev_remote) as tracker:
    qnode_remote(params)

Let's take a look at what new fields are available in the tracker with an on-demand simulator.

In [13]:
for key in tracker.history.keys():
    print(key)

executions
shots
braket_task_id
braket_simulator_ms
braket_simulator_billed_ms
batches
batch_len


The `braket_task_id` field will contain unique IDs for each task that was created.
For tasks that are created on the on-demand simulators, these IDs can be recorded and used to look up the task details in the AWS console.
The `braket_simulator_ms` gives the duration in milliseconds of the portion of the task spent simulating our circuit.
`braket_simulator_billed_ms` adjusts the simulation duration according to the minimum duration for billing.
See [here](https://aws.amazon.com/braket/pricing/) for pricing details on Amazon Braket.

These reported times are not the total time for a task to complete.
We can compare the simulation time to the total task time for this circuit:

In [14]:
with qml.Tracker(dev_remote) as tracker:
    t0 = time.time()
    qnode_remote(params)
    t1 = time.time()
print("Remote device execution duration in seconds:",t1 - t0)
print("Simulation duration in seconds:", tracker.totals["braket_simulator_ms"] / 1000)

Remote device execution duration in seconds: 3.1985151767730713
Simulation duration in seconds: 0.012


With such a small circuit in this example, only a very small fraction of the total time is spent running the simulation of the circuit.

## Using the Tracker to Limit Resource Usage

As we have seen above, it is possible to record usage of a device during execution with the resource tracker.
With this information available during the computation, it is possible to handle the information in order to control the resource usage.
However, this feature does not attempt to estimate how many resources may be used in the future. It is purely backward-looking.
Thus any resource usage can only be acted on after the fact with this feature.

In the [PennyLane getting started notebook](../0_Getting_started/0_Getting_started.ipynb),
we optimized a circuit by running the gradient descent optimizer for a chosen number of iterations.
Instead of a given number of iterations, we may want to limit the optimization to something else such as the total number of shots or the simulator execution time.
Here we will show two ways to use the PennyLane device tracker to limit an optimization on a resource which is being tracked.

Let's suppose we wish to optimize our simple circuit, but want to limit ourselves to a particular number of circuit executions.
First let's set up an optimizer.

In [15]:
opt = qml.GradientDescentOptimizer(stepsize=0.2)

Now we run the optimizer, but on every step of the loop we check if our limit has been crossed.
Once the total executions exceed our target limit, we will break out of the optimization and report our results.

In [16]:
max_iterations = 100
execution_limit = 77
params = np.array([0.1, 0.2], requires_grad=True)

with qml.Tracker(dev) as tracker:
    for i in range(max_iterations):
        params, cost = opt.step_and_cost(qnode_local, params)
        if tracker.totals["executions"] > execution_limit:
            break

print("Completed", i + 1, "steps of optimization.")
print("Optimized cost:", cost)
print("Optimized parameters:", params)


Completed 16 steps of optimization.
Optimized cost: -0.27959738151720537
Optimized parameters: [0.51754609 2.06622178]


The resource usage is compared only after each iteration of the optimization.
If each of the steps in your computation involve many circuit evaluations,
your computation may still continue for many evaluations after your limit is hit.
For our simple problem, every iteration leads to 5 circuit evaluations, so we do not stop right at our limit.
We can see this by checking the tracker totals.

In [17]:
print(tracker.totals)

{'executions': 80, 'batches': 32, 'batch_len': 80}


Take this into consideration when setting a threshold if you choose to limit your computation this way.

For another approach to limit resource usage, we can add a callback function to the device tracker.
This function is called each time the information in the tracker is updated.
This interface could also be used for monitoring large batches of executions.
For example, let's print out the running total number of executions every time the tracker is updated.

In [ ]:
def log_executions(totals, history, latest):
    print("Total executions:",totals["executions"])

with qml.Tracker(dev, callback=log_executions) as tracker:
    qml.grad(qnode_local)(params)

Total executions: 1
Total executions: 1
Total executions: 2
Total executions: 3
Total executions: 4
Total executions: 5
Total executions: 5


We can use the callback feature of the device tracker to stop a computation after a threshold is breached.
By throwing an exception from our callback function when the new data violates our limit, we can abort the currently running optimization.
Let's try this approach with the same optimization problem.

In [19]:
params = np.array([0.1, 0.2], requires_grad=True)

def resource_threshold(totals, history, latest):
    if totals["executions"] > execution_limit:
        raise ResourceWarning()

with qml.Tracker(dev, callback=resource_threshold) as tracker:
    try:
        for i in range(max_iterations):
            params,cost = opt.step_and_cost(qnode_local, params)
    except ResourceWarning:
        print("Completed",i,"steps of optimization.")
        print("Optimized cost:", cost)
        print("Optimized parameters:", params)



Completed 15 steps of optimization.
Optimized cost: -0.14125448783850006
Optimized parameters: [0.5519834  1.90536732]


This time one fewer optimization step is completed because the final step is aborted.
We can look at the totals to see that the execution was stopped after reaching our set limit.

In [20]:
print(tracker.totals)

{'executions': 78, 'batches': 31, 'batch_len': 76}


**Note** This approach will immediately terminate the optimization upon exceeding the resource limit.
If the termination happens while in the middle of a step, that step will not finish.
In this case, circuits will have been executed that will then have their results discarded.

## Summary

In this notebook, we have seen the resource usage recorded by the PennyLane device tracker and the extended information provided by Amazon Braket.
We showed two different ways to use the resource information in the middle of the computation, and we were able to control the optimization routine to limit usage according to a preset threshold.
If you are looking to explore more, try modifying the [QAOA notebook](../2_Graph_optimization_with_QAOA/2_Graph_optimization_with_QAOA.ipynb) to track the use of the simulators in that example.